In [ ]:
import os
import pandas as pd

def generate_visual_tree(startpath):
    """
    指定されたパスのフォルダ構造をスキャンし、
    ツリー表示用の文字列リストを生成して返す関数
    """
    #Excelシート分割用のデータ収集辞書を初期化
    folder_data = {}
    # インデント用の文字列
    space = '    '
    pipe_space = '│' + space
    #パスが存在しない場合は処理を中断し、エラーメッセージを返す
    if not os.path.isdir(startpath):
        return[f'指定されたフォルダ({startpath})が見つからないか、フォルダではありません。スキップします。']
    #ルートフォルダの文字列を生成
    root_key = os.path.basename(startpath) + os.sep
    #ルートフォルダ配下のツリー表示用のリストの初期化
    root_lines = []
    #ルートフォルダのパスを一行目に追加
    root_lines.append(root_key)

    #ルートフォルダ直下のファイル・ディレクトリを取得
    root_items = os.listdir(startpath)
    #フルパスからファイルのみ取得、リスト化
    root_files = [
        item for item in root_items
        if os.path.isfile(os.path.join(startpath, item))
    ]
    #ファイル名でソート
    root_files.sort()
    #ルートフォルダ直下にサブディレクトリが存在するか確認
    has_subdirs = any(os.path.isdir(os.path.join(startpath, item)) for item in root_items)
    #ルートフォルダ直下のファイルにインデックスを付与してツリー表示用の文字列を生成
    for i, file in enumerate(root_files):
        # ルートフォルダ直下の最後のファイルかどうかを判定(True/False)
        is_last_file = (i == len(root_files) - 1)
        #ルートフォルダの直下にサブフォルダがない、かつ最後のファイルの場合は'└── 'を使用
        if is_last_file and not has_subdirs:
            root_prefix = '└── '
        #それ以外の場合は'├── 'を使用
        else:
            root_prefix = '├── '
        #ツリー表示用の文字列を生成してリストに追加
        root_lines.append(f'{root_prefix}📄{file}')
    #ルートフォルダのデータを辞書に追加(Excelのルートシート用)
    folder_data[root_key] = root_lines
    #os.walkでサブフォルダ以下の全階層を巡回
    for root, dirs, files in os.walk(startpath):
        # ルートフォルダの場合、サブフォルダ名を辞書に追加してスキップ
        if root == startpath:
            for dir_name in dirs:
                #folder_dataにサブフォルダ名をkeyとして追加
                folder_data[dir_name + os.sep] = [dir_name + os.sep]
            continue
        #ディレクトリ・ファイルをアルファベット順にソート
        dirs.sort()
        files.sort()

        #ルートフォルダからの相対パスを取得
        root_relative = root.replace(startpath + os.sep, '')
        #Excelシートのキーとなるトップレベルディレクトリ名の取得
        top_level_dir = root_relative.split(os.sep)[0] + os.sep
        #トップレベルフォルダが有効なシートキーとして存在するか確認
        if top_level_dir in folder_data:
            current_tree_list = folder_data[top_level_dir]
        else:
            continue
        #ルートからの相対的な階層の深さ(区切り文字数)を計算
        level = root_relative.count(os.sep)
        #Excelシートの起点に合わせ、階層レベルを調整(‐１)してインデントを計算
        indent_base = pipe_space * (level - 1)
        #現在処理中のフォルダ名(パス区切り文字を含まない)を取得
        dir_name = os.path.basename(root)
        
        is_last_dir_in_list = dir_name == (dirs[-1] if dirs else dir_name)

        if is_last_dir_in_list:
            folder_prefix = '└── '
            next_indent = space
        else:
            folder_prefix = '├── '
            next_indent = pipe_space
        
        current_tree_list.append(f'{indent_base}{folder_prefix}📂{dir_name}{os.sep}')

        file_indent_base = indent_base + next_indent

        for i, file in enumerate(files):
            is_last_file = (i == len(files) - 1)
            if is_last_file:
                file_prefix = '└── '
            else:
                file_prefix = '├── '

            tree_lines.append(f'{file_indent_base}{file_prefix}📄{file}')

    return tree_lines